In [24]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable First Segment

In [2]:
# Load and inspect the data.csv dataset
file = 'data.csv'
spotify_df = pd.read_csv(file)
spotify_df.head(5)

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.995,['Carl Woitschach'],0.708,158648,0.1950,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928
1,0.994,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,0,6KuQTIu1KoTTkLXKrwlLPV,0.901,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,0.0462,83.972,0.0767,1928
2,0.604,['Seweryn Goszczyński'],0.749,104300,0.2200,0,6L63VW0PibdM1HDSBoqnoM,0.000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928
3,0.995,['Francisco Canaro'],0.781,180760,0.1300,0,6M94FkXd15sOAOQYRnWPN8,0.887,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.7200,1928
4,0.990,"['Frédéric Chopin', 'Vladimir Horowitz']",0.210,687733,0.2040,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928


In [3]:
spotify_df.shape

(169909, 19)

In [4]:
spotify_df.describe()

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,year
count,169909.000000,169909.000000,1.699090e+05,169909.000000,169909.000000,169909.000000,169909.000000,169909.000000,169909.000000,169909.000000,169909.000000,169909.000000,169909.000000,169909.000000,169909.000000
mean,0.493214,0.538150,2.314062e+05,0.488593,0.084863,0.161937,5.200519,0.206690,-11.370289,0.708556,31.556610,0.094058,116.948017,0.532095,1977.223231
std,0.376627,0.175346,1.213219e+05,0.267390,0.278679,0.309329,3.515257,0.176796,5.666765,0.454429,21.582614,0.149937,30.726937,0.262408,25.593168
min,0.000000,0.000000,5.108000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1921.000000
25%,0.094500,0.417000,1.710400e+05,0.263000,0.000000,0.000000,2.000000,0.098400,-14.470000,0.000000,12.000000,0.034900,93.516000,0.322000,1957.000000
50%,0.492000,0.548000,2.086000e+05,0.481000,0.000000,0.000204,5.000000,0.135000,-10.474000,1.000000,33.000000,0.045000,114.778000,0.544000,1978.000000
75%,0.888000,0.667000,2.629600e+05,0.710000,0.000000,0.086800,8.000000,0.263000,-7.118000,1.000000,48.000000,0.075400,135.712000,0.749000,1999.000000
max,0.996000,0.988000,5.403500e+06,1.000000,1.000000,1.000000,11.000000,1.000000,3.855000,1.000000,100.000000,0.969000,244.091000,1.000000,2020.000000


In [5]:
# confirm the data type
spotify_df.dtypes

acousticness        float64
artists              object
danceability        float64
duration_ms           int64
energy              float64
explicit              int64
id                   object
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
name                 object
popularity            int64
release_date         object
speechiness         float64
tempo               float64
valence             float64
year                  int64
dtype: object

In [6]:
# Find null values
for column in spotify_df.columns:
    print(f"Column {column} has {spotify_df[column].isnull().sum()}null values")

Column acousticness has 0null values
Column artists has 0null values
Column danceability has 0null values
Column duration_ms has 0null values
Column energy has 0null values
Column explicit has 0null values
Column id has 0null values
Column instrumentalness has 0null values
Column key has 0null values
Column liveness has 0null values
Column loudness has 0null values
Column mode has 0null values
Column name has 0null values
Column popularity has 0null values
Column release_date has 0null values
Column speechiness has 0null values
Column tempo has 0null values
Column valence has 0null values
Column year has 0null values


In [7]:
# Check for duplicates
print(f"Duplicated entries: {spotify_df.duplicated().sum()}")

Duplicated entries: 0


In [8]:
# string to int
spotify_df.set_index('id', inplace = True)
spotify_df.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
id,,,,,,,,,,,,,,,,,,
6KbQ3uYMLKb5jDxLF7wYDD,0.995,['Carl Woitschach'],0.708,158648,0.1950,0,0.563,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928
6KuQTIu1KoTTkLXKrwlLPV,0.994,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,0,0.901,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,0.0462,83.972,0.0767,1928
6L63VW0PibdM1HDSBoqnoM,0.604,['Seweryn Goszczyński'],0.749,104300,0.2200,0,0.000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928
6M94FkXd15sOAOQYRnWPN8,0.995,['Francisco Canaro'],0.781,180760,0.1300,0,0.887,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.7200,1928
6N6tiFZ9vLTSOIxkj8qKrd,0.990,"['Frédéric Chopin', 'Vladimir Horowitz']",0.210,687733,0.2040,0,0.908,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928


In [9]:
# Create a new DataFrame that holds the string columns.
song_names_df = spotify_df[["name", "artists", "release_date", "year", "duration_ms"]].copy()
song_names_df 

,name,artists,release_date,year,duration_ms
id,,,,,
6KbQ3uYMLKb5jDxLF7wYDD,Singende Bataillone 1. Teil,['Carl Woitschach'],1928,1928,158648
6KuQTIu1KoTTkLXKrwlLPV,"Fantasiestücke, Op. 111: Più tosto lento","['Robert Schumann', 'Vladimir Horowitz']",1928,1928,282133
6L63VW0PibdM1HDSBoqnoM,Chapter 1.18 - Zamek kaniowski,['Seweryn Goszczyński'],1928,1928,104300
6M94FkXd15sOAOQYRnWPN8,Bebamos Juntos - Instrumental (Remasterizado),['Francisco Canaro'],1928-09-25,1928,180760
6N6tiFZ9vLTSOIxkj8qKrd,"Polonaise-Fantaisie in A-Flat Major, Op. 61","['Frédéric Chopin', 'Vladimir Horowitz']",1928,1928,687733
...,...,...,...,...,...
4KppkflX7I3vJQk7urOJaS,Skechers (feat. Tyga) - Remix,"['DripReport', 'Tyga']",2020-05-15,2020,163800
1ehhGlTvjtHo2e4xJFB0SZ,Sweeter (feat. Terrace Martin),"['Leon Bridges', 'Terrace Martin']",2020-06-08,2020,167468
52eycxprLhK3lPcRLbQiVk,How Would I Know,"['Kygo', 'Oh Wonder']",2020-05-29,2020,180700


In [13]:
# Drop the string columns since it's not going to be used on the clustering algorithm.
spotify_df = spotify_df.drop(columns = ["name", "artists", "release_date", "year", "duration_ms"])
spotify_df

,acousticness,danceability,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
id,,,,,,,,,,,,,
6KbQ3uYMLKb5jDxLF7wYDD,0.9950,0.708,0.1950,0,0.563000,10,0.1510,-12.428,1,0,0.0506,118.469,0.7790
6KuQTIu1KoTTkLXKrwlLPV,0.9940,0.379,0.0135,0,0.901000,8,0.0763,-28.454,1,0,0.0462,83.972,0.0767
6L63VW0PibdM1HDSBoqnoM,0.6040,0.749,0.2200,0,0.000000,5,0.1190,-19.924,0,0,0.9290,107.177,0.8800
6M94FkXd15sOAOQYRnWPN8,0.9950,0.781,0.1300,0,0.887000,1,0.1110,-14.734,0,0,0.0926,108.003,0.7200
6N6tiFZ9vLTSOIxkj8qKrd,0.9900,0.210,0.2040,0,0.908000,11,0.0980,-16.829,1,1,0.0424,62.149,0.0693
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4KppkflX7I3vJQk7urOJaS,0.1730,0.875,0.4430,1,0.000032,1,0.0891,-7.461,1,75,0.1430,100.012,0.3060
1ehhGlTvjtHo2e4xJFB0SZ,0.0167,0.719,0.3850,0,0.031300,8,0.1110,-10.907,1,64,0.0403,128.000,0.2700
52eycxprLhK3lPcRLbQiVk,0.5380,0.514,0.5390,0,0.002330,7,0.1080,-9.332,1,70,0.1050,123.700,0.1530


In [25]:
spotify_df.loc[spotify_df.danceability >= 0.5, 'danceability'] = 1
spotify_df.loc[spotify_df.danceability < 0.5, 'danceability'] = 0
spotify_df["danceability"].value_counts()

1.0    102122
0.0     67787
Name: danceability, dtype: int64

In [26]:
spotify_df.head()

,acousticness,danceability,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,class
id,,,,,,,,,,,,,,
6KbQ3uYMLKb5jDxLF7wYDD,0.995,1.0,0.1950,0,0.563,10,0.1510,-12.428,1,0,0.0506,118.469,0.7790,1
6KuQTIu1KoTTkLXKrwlLPV,0.994,0.0,0.0135,0,0.901,8,0.0763,-28.454,1,0,0.0462,83.972,0.0767,1
6L63VW0PibdM1HDSBoqnoM,0.604,1.0,0.2200,0,0.000,5,0.1190,-19.924,0,0,0.9290,107.177,0.8800,1
6M94FkXd15sOAOQYRnWPN8,0.995,1.0,0.1300,0,0.887,1,0.1110,-14.734,0,0,0.0926,108.003,0.7200,1
6N6tiFZ9vLTSOIxkj8qKrd,0.990,0.0,0.2040,0,0.908,11,0.0980,-16.829,1,1,0.0424,62.149,0.0693,1


### Split the Data into Training and Testing

In [27]:
# Create our features
X = pd.get_dummies(spotify_df.drop('danceability', axis=1))

# Create our target
y = spotify_df["danceability"]

In [28]:
X.describe()

,acousticness,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,class
count,169909.000000,169909.000000,169909.000000,169909.000000,169909.000000,169909.000000,169909.000000,169909.000000,169909.000000,169909.000000,169909.000000,169909.000000,169909.000000
mean,0.493214,0.488593,0.084863,0.161937,5.200519,0.206690,-11.370289,0.708556,31.556610,0.094058,116.948017,0.532095,1.070544
std,0.376627,0.267390,0.278679,0.309329,3.515257,0.176796,5.666765,0.454429,21.582614,0.149937,30.726937,0.262408,0.827752
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.094500,0.263000,0.000000,0.000000,2.000000,0.098400,-14.470000,0.000000,12.000000,0.034900,93.516000,0.322000,0.000000
50%,0.492000,0.481000,0.000000,0.000204,5.000000,0.135000,-10.474000,1.000000,33.000000,0.045000,114.778000,0.544000,1.000000
75%,0.888000,0.710000,0.000000,0.086800,8.000000,0.263000,-7.118000,1.000000,48.000000,0.075400,135.712000,0.749000,2.000000
max,0.996000,1.000000,1.000000,1.000000,11.000000,1.000000,3.855000,1.000000,100.000000,0.969000,244.091000,1.000000,2.000000


In [29]:
# Check the balance of our target values
y.value_counts()

1.0    102122
0.0     67787
Name: danceability, dtype: int64

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(127431, 13)

In [35]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [36]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_resampled, y_resampled)

/Users/dianaescobar/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [37]:
y_pred = model.predict(X_test)

from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.7642345964526348